In [1]:
import quimb.tensor as qtn
import quimb as qu
import numpy as np
import pickle
import scipy.sparse.linalg as spla
from utils import progress_bar
from mpo_utils import mps_to_mpo

/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:57: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:39: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:76: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


In [2]:
def get_rho_conj(rho,upper_ind_id='b{}',lower_ind_id='a{}',tags='rho*'):
    site_tag_len=len(rho.site_tag_id)-2
    ind_order=[int(t[site_tag_len:]) for t in rho.tags if t in rho.site_tags]
    
    tens=[rho.arrays[j].conj() for j in [ind_order.index(i) for i in range(rho.L)]]
    
    rho_conj=qtn.tensor_1d.MatrixProductOperator(tens,
                                                upper_ind_id=upper_ind_id,
                                                lower_ind_id=lower_ind_id,
                                                tags=tags,shape='lrud') #conjugate the operator
 
    return rho_conj

def mirror_mpo(mpo):
    mpo.permute_arrays(shape='lrud')
    mirrored_arrays = [mpo.arrays[-1]]
    for arr in list(reversed(mpo.arrays))[1:-1]:
        mirrored_arrays.append(arr.transpose(1,0,2,3))
    mirrored_arrays.append(mpo.arrays[0])
    return qtn.tensor_1d.MatrixProductOperator(mirrored_arrays)


In [3]:


def project_out(rho,rho_conj,rho0,i,lix,rix):
    '''
      _____
      |a  |tensors_sorted
      _____
      |c  |
    [r]  |
      |d  |
    [r0*] |
      |c__|

    ''' 
    site_tag_len=len(rho0.site_tag_id)-2
    ind_order=[int(t[site_tag_len:]) for t in rho0.tags if t in rho0.site_tags]
    
    tens=[rho0.arrays[j] for j in [ind_order.index(i) for i in range(rho0.L)]]
    rho0=qtn.MatrixProductOperator(tens,
                                    upper_ind_id='b{}', lower_ind_id='a{}',
                                    tags='rho0',shape='lrud')
    
    rho0_conj=get_rho_conj(rho0,upper_ind_id='c{}',lower_ind_id='d{}',tags='rho0*')
    

    

    # Normalize all MPOs
    
    full_network=qtn.TensorNetwork1D((rho,rho0,rho_conj,rho0_conj))

    
    print((rho0_conj*(1/rho0_conj.norm())&rho*(1/rho.norm())).contract())
    full_network._L=rho.L
    full_network._site_tag_id='I{}'
    menv=qtn.tensor_dmrg.MovingEnvironment(full_network,begin='left',bsz=2)
    menv.move_to(i)
    curnet = menv()
    linop_tens=[curnet['rho0*',f'I{i}'],
                curnet['rho0*',f'I{i+1}'],
                curnet['rho0',f'I{i}'],
                curnet['rho0',f'I{i+1}'],
                curnet['_LEFT'],
                curnet['_RIGHT']]
    return qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')

In [4]:
#Loading the data ( half of the system)
folder='DATA/new_data02_L6/'
with open(folder+'sca_xs.pkl', 'rb') as file:
    sca_xs = pickle.load(file)
with open(folder+'sca_zs.pkl', 'rb') as file:
    sca_zs = pickle.load(file)
with open(folder+'scd_xs.pkl', 'rb') as file:
    scd_xs = pickle.load(file)
with open(folder+'scd_zs.pkl', 'rb') as file:
    scd_zs = pickle.load(file)

#Mirror the data to get the full system
sca_xs_mirrored = [mirror_mpo(sca_x) for sca_x in reversed(sca_xs)]
sca_zs_mirrored = [mirror_mpo(sca_z) for sca_z in reversed(sca_zs)]
scd_xs_mirrored = [mirror_mpo(scd_x) for scd_x in reversed(scd_xs)]
scd_zs_mirrored = [mirror_mpo(scd_z) for scd_z in reversed(scd_zs)]

scas=sca_xs+sca_xs_mirrored+sca_zs+sca_zs_mirrored
scds=scd_xs+scd_xs_mirrored+scd_zs+scd_zs_mirrored
L=scas[0].L



In [5]:
scd_zs

[MatrixProductOperator(tensors=6, indices=17, L=6, max_bond=8),
 MatrixProductOperator(tensors=6, indices=17, L=6, max_bond=35),
 MatrixProductOperator(tensors=6, indices=17, L=6, max_bond=55)]

In [6]:
sum([np.prod(x.shape) for x in scd_zs[-1]])/2**(2*L)

np.float64(1.8515625)

In [7]:
h=1.5
J=1
L=scas[0].L
print(L)
split_opts={'method':'svd','cutoff':1e-10,'cutoff_mode':'rel','max_bond':100}
h2=-J*(qu.pauli('Z') & qu.pauli('I')) & (qu.pauli('Z') & qu.pauli('I'))
h1=h*qu.pauli('X') & qu.pauli('I')
local_ham = qtn.LocalHam1D(L=L, H2=h2, H1=h1,cyclic=False)
def gibbs_mpo(local_ham,tebd_tol,split_opts):
    rho0=qtn.MPS_product_state([np.eye(2)]*L)
    tebd=qtn.TEBD(rho0,local_ham,imag=True,progbar=False,split_opts=split_opts)
    tebd.update_to(.5,tol=tebd_tol)
    rho=mps_to_mpo(tebd.pt)
    rho=rho*np.sqrt(1/(rho.apply(rho)).trace())
    return rho
rho0=gibbs_mpo(local_ham,1e-10,split_opts)

6


In [8]:
'''
 _____
 |a  |
[r*] |
 |b  |
[lm] |
 |c  |
[r]  |
 |d  |
[rm] |
 |a__|

'''

'lrud'
#initialize the density matrix
rho=sca_xs[0].identity().left_canonize()
#construct the discriminant: Disc = Sc[A][.]Sc[A]^dag-Sc[D][.]-[.]Sc[D]
lscds=[-1*scd.copy() for scd in scds]
rscds=[-1*scd.copy() for scd in scds]
lscas=[sca.copy() for sca in scas]
rscas=[sca.copy() for sca in scas]

left_mpos=lscas+lscds+[scds[0].identity()]*len(lscds)
right_mpos=rscas+[scds[0].identity()]*len(lscds)+rscds

length=rho.L

rho=qtn.MatrixProductOperator(rho.arrays, upper_ind_id='c{}', lower_ind_id='d{}', tags='rho',shape='lrud')


In [9]:
rho.randomize()

MatrixProductOperator(tensors=6, indices=17, L=6, max_bond=1)

In [10]:
rho_conj=get_rho_conj(rho) 

new_left_mpos=[]
new_right_mpos=[]
for i in range(len(left_mpos)):
    left_mpo=qtn.MatrixProductOperator(left_mpos[i].arrays, upper_ind_id='b{}', lower_ind_id='c{}', tags='lmpo',shape='lrud')
    right_mpo=qtn.MatrixProductOperator(right_mpos[i].arrays, upper_ind_id='d{}', lower_ind_id='a{}', tags='rmpo',shape='lrud')
    new_left_mpos.append(left_mpo)
    new_right_mpos.append(right_mpo)
right_mpos=new_right_mpos
left_mpos=new_left_mpos


In [11]:

new_left_mpos=[]
new_right_mpos=[]
for i in range(len(left_mpos)):
    left_mpo=qtn.MatrixProductOperator(left_mpos[i].arrays, upper_ind_id='b{}', lower_ind_id='c{}', tags='lmpo',shape='lrud')
    right_mpo=qtn.MatrixProductOperator(right_mpos[i].arrays, upper_ind_id='d{}', lower_ind_id='a{}', tags='rmpo',shape='lrud')
    new_left_mpos.append(left_mpo)
    new_right_mpos.append(right_mpo)
right_mpos=new_right_mpos
left_mpos=new_left_mpos


In [12]:
menvs=[]
for k in range(len(left_mpos)):
    full_network=qtn.TensorNetwork1D((rho,rho_conj,left_mpos[k],right_mpos[k]))
    full_network._L=rho.L
    full_network._site_tag_id='I{}'
    menvs.append(qtn.tensor_dmrg.MovingEnvironment(full_network,begin='right',bsz=2))

i=0
v0 = (rho[f'I{i}'] & rho[f'I{i+1}']) # get the double site to optimize
v0c=v0.contract()
# get the indicies of the linear operator
rix = v0c.inds
lix = (rho_conj[f'I{i}'] & rho_conj[f'I{i+1}']).outer_inds() 
v_shape=v0c.shape
#get all the linear operators from the left and tight mpos
for j,menv in enumerate(menvs):
    menv.move_to(i)
    curnet = menv()
    linop_tens=[curnet['lmpo',f'I{i}'],
                curnet['lmpo',f'I{i+1}'],
                curnet['rmpo',f'I{i}'],
                curnet['rmpo',f'I{i+1}'],
                curnet['_LEFT'],
                curnet['_RIGHT']]
    if j==0:
        linop=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
        tn=qtn.tensor_core.TensorNetwork(linop_tens).contract()
    else:
        linop+=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
        tn+=qtn.tensor_core.TensorNetwork(linop_tens).contract()
        
    mat=tn.transpose(*lix,*rix).data.reshape(np.prod(v_shape),np.prod(v_shape))



#find the eigenvector of the full linear operator
_,evecs=spla.eigsh(linop,k=1,which=which_eval,v0=v0c.data.reshape(-1),tol=10**(-5))
#split the linear operator into two sites
combined_sites=qtn.Tensor(evecs.T[0].reshape(v_shape),inds=rix)
if i==rho.L-2:
    split_inds=rix[:int(len(rix)/2+0.5)]
else:
    split_inds=rix[:int(len(rix)/2)]
new_sites=combined_sites.split(left_inds=split_inds,
                               absorb='right',
                               rtags=['rho',f'I{i+1}'],
                               ltags=['rho',f'I{i}'],
                               bond_ind=v0.inner_inds()[0],
                               **split_opts)
new_rho1=new_sites.tensors[0]
new_rho2=new_sites.tensors[1]
#replace the sites in the density mpo rho
rho[f'I{i}']=new_rho1.transpose(*rho[f'I{i}'].inds)
rho[f'I{i+1}']=new_rho2.transpose(*rho[f'I{i+1}'].inds)
rho_conj=get_rho_conj(rho)
#update the menvs
menvs=[]
for k in range(len(left_mpos)):
    full_network=qtn.TensorNetwork1D((rho,rho_conj,left_mpos[k],right_mpos[k]))
    full_network._L=rho.L
    full_network._site_tag_id='I{}'
    menvs.append(qtn.tensor_dmrg.MovingEnvironment(full_network,begin='left',bsz=2))

In [13]:
which_eval='LA'
n=0
N=4
while n<N:
    tol=10**(-2*n-1)
    split_opts = {'method': 'svd','cutoff_mode':'abs', 'max_bond': 100, 'cutoff': tol}
    print('\nright_sweep: ',n)
    
    for i in range(rho.L-2):
        progress_bar(i,rho.L-1)
        v0 = (rho[f'I{i}'] & rho[f'I{i+1}']) # get the double site to optimize
        v0c=v0.contract()
        # get the indicies of the linear operator
        rix = v0c.inds
        lix = (rho_conj[f'I{i}'] & rho_conj[f'I{i+1}']).outer_inds() 
        v_shape=v0c.shape
        #get all the linear operators from the left and tight mpos
        linop=-1*project_out(rho,rho_conj,rho0,i,lix,rix)
        for j,menv in enumerate(menvs):
            menv.move_to(i)
            curnet = menv()
            linop_tens=[curnet['lmpo',f'I{i}'],
                        curnet['lmpo',f'I{i+1}'],
                        curnet['rmpo',f'I{i}'],
                        curnet['rmpo',f'I{i+1}'],
                        curnet['_LEFT'],
                        curnet['_RIGHT']]
            linop+=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            '''
            if j==0:
                linop=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            else:
                linop+=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            '''
        #find the eigenvector of the full linear operator
        _,evecs=spla.eigsh(linop,k=1,which=which_eval,v0=v0c.data.reshape(-1),tol=tol)
        #_,evecs=spla.lobpcg(linop,largest=True,X=v0c.data.reshape(-1,1),tol=10**(-3))
        #split the linear operator into two sites
        res_vec=evecs.T[0]
        combined_sites=qtn.Tensor(res_vec.reshape(v_shape),inds=rix)
        
        if i==rho.L-2:
            split_inds=rix[:int(len(rix)/2+0.5)]
        else:
            split_inds=rix[:int(len(rix)/2)]
        new_sites=combined_sites.split(left_inds=split_inds,
                                    absorb='right',
                                    rtags=['rho',f'I{i+1}'],
                                    ltags=['rho',f'I{i}'],
                                    bond_ind=v0.inner_inds()[0],
                                    **split_opts)
        new_rho1=new_sites.tensors[0]
        new_rho2=new_sites.tensors[1]
        #replace the sites in the density mpo rho
        rho[f'I{i}']=new_rho1.transpose(*rho[f'I{i}'].inds)
        rho[f'I{i+1}']=new_rho2.transpose(*rho[f'I{i+1}'].inds)
        rho_conj=get_rho_conj(rho)
        #update the menvs
        menvs=[]
        for k in range(len(left_mpos)):
            full_network=qtn.TensorNetwork1D((rho,rho_conj,left_mpos[k],right_mpos[k]))
            full_network._L=rho.L
            full_network._site_tag_id='I{}'
            menvs.append(qtn.tensor_dmrg.MovingEnvironment(full_network,begin='left',bsz=2))
    print('\nleft_sweep: ',n) 
    for i in reversed(list(range(1,rho.L-1))):
        progress_bar(i,rho.L-1)
        v0 = (rho[f'I{i}'] & rho[f'I{i+1}']) # get the double site to optimize
        v0c=v0.contract()
        # get the indicies of the linear operator
        rix = v0c.inds
        lix = (rho_conj[f'I{i}'] & rho_conj[f'I{i+1}']).outer_inds() 
        v_shape=v0c.shape
        linop=-1*project_out(rho,rho_conj,rho0,i,lix,rix)
        for j,menv in enumerate(menvs):
            menv.move_to(i)
            curnet = menv()
            linop_tens=[curnet['lmpo',f'I{i}'],
                        curnet['lmpo',f'I{i+1}'],
                        curnet['rmpo',f'I{i}'],
                        curnet['rmpo',f'I{i+1}'],
                        curnet['_LEFT'],
                        curnet['_RIGHT']]
            linop+=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            '''
            if j==0:
                linop=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            else:
                linop+=qtn.tensor_core.TNLinearOperator(linop_tens,left_inds=lix,right_inds=rix,optimize='auto-hq')
            '''
        
        _,evecs=spla.eigsh(linop,k=1,which=which_eval,v0=v0c.data.reshape(-1),tol=tol)
        #_,evecs=spla.lobpcg(linop,largest=True,X=v0c.data.reshape(-1,1),tol=10**(-3))
        #evecs= qu.linalg.base_linalg.eigvecsh()
        res_vec=evecs.T[0]
        combined_sites=qtn.Tensor(res_vec.reshape(v_shape),inds=rix)
        if i==rho.L-2:
            split_inds=rix[:int(len(rix)/2+0.5)]
        else:
            split_inds=rix[:int(len(rix)/2)]
        new_sites=combined_sites.split(left_inds=split_inds,absorb='left',rtags=[f'I{i+1}','rho'],ltags=[f'I{i}','rho'],bond_ind=v0.inner_inds()[0],**split_opts)
        new_rho1=new_sites.tensors[0]
        new_rho2=new_sites.tensors[1]
        rho[f'I{i}']=new_rho1.transpose(*rho[f'I{i}'].inds)
        rho[f'I{i+1}']=new_rho2.transpose(*rho[f'I{i+1}'].inds)
        rho_conj=get_rho_conj(rho)
        menvs=[]
        for k in range(len(left_mpos)):
            trace=rho.trace()
            full_network=qtn.TensorNetwork1D((rho,rho_conj,left_mpos[k],right_mpos[k]))
            full_network._L=rho.L
            full_network._site_tag_id='I{}'
            menvs.append(qtn.tensor_dmrg.MovingEnvironment(full_network,begin='right',bsz=2))
    eval=sum([menv().contract(all) for menv in menvs])
    print('\n',eval)
    n+=1



right_sweep:  0
0%|□□□□□□□□□□□□□□□□□□□□|0.3442735206317318
25%|■■■■■□□□□□□□□□□□□□□□|(0.47990501947162156-3.2459607167847068e-12j)
50%|■■■■■■■■■■□□□□□□□□□□|(2.978920278853825e-12+2.9563023903889096e-11j)
75%|■■■■■■■■■■■■■■■□□□□□|(5.422719565051359e-13+5.399507230219314e-12j)

left_sweep:  0
100%|■■■■■■■■■■■■■■■■■■■■|(4.099255657141754e-13+4.092053085269498e-12j)
75%|■■■■■■■■■■■■■■■□□□□□|(3.588525922747637e-13+3.5920916996903175e-12j)
50%|■■■■■■■■■■□□□□□□□□□□|(3.195443932921102e-13+3.1986022281624934e-12j)
25%|■■■■■□□□□□□□□□□□□□□□|(2.681633195077386e-13+2.683423793711246e-12j)

 -0.2568201055995988

right_sweep:  1
0%|□□□□□□□□□□□□□□□□□□□□|(1.224229787087701e-13+1.2138134045455574e-12j)
25%|■■■■■□□□□□□□□□□□□□□□|(1.22192240069641e-13+1.2113018667503118e-12j)
50%|■■■■■■■■■■□□□□□□□□□□|(1.2130865890606735e-13+1.2028365605784386e-12j)
75%|■■■■■■■■■■■■■■■□□□□□|(1.2379348979485558e-13+1.2307488002088854e-12j)

left_sweep:  1
100%|■■■■■■■■■■■■■■■■■■■■|(1.2199608892970798e-13+1.2145804890766258e-

In [ ]:
[-0.2881276519371354, -0.2400985759516535,-0.218154438689567]

[-0.2881276519371354]

In [ ]:
np.linalg.norm(rho0.to_dense()-rho.to_dense(),ord=2)/np.linalg.norm(rho0.to_dense(),ord=2)

np.float64(1.3643463975428007)

In [16]:
left_mpos=lscas+lscds+[scds[0].identity()]*len(lscds)
right_mpos=rscas+[scds[0].identity()]*len(lscds)+rscds

par=left_mpos[0].apply(rho,compress=True)
res=par.apply(right_mpos[0],compress=True)
for left_mpo,right_mpo in zip(left_mpos[1:],right_mpos[1:]):
    par=left_mpo.apply(rho,compress=True)
    res+=par.apply(right_mpo,compress=True)



SystemError: CPUDispatcher(<function qr_stabilized_numba at 0x7f27c39d3a60>) returned a result with an exception set

In [17]:

res_mat=res.to_dense()
rho_mat=rho.to_dense()

((res_mat/res_mat.trace())-(rho_mat/rho_mat.trace())).trace()

np.complex128(-9.5367431640625e-06j)

In [18]:
import pickle
import os

# Save the rho variable into the debug folder
# Create the debug folder if it does not exist
os.makedirs('debug', exist_ok=True)
with open('debug/rho.pkl', 'wb') as file:
    pickle.dump(rho, file)

In [19]:

eval=sum([menv().contract(all) for menv in menvs])
eval

-0.2881276519371354